##Importando bibliotecas

In [108]:
import pandas as pd
import requests
import kagglehub #
import os
import numpy as np
import re
import sqlite3


## Download do Dataset Disney

In [109]:

# Download do Dataset Dysney Movies and TV Shows

path = kagglehub.dataset_download("shivamb/disney-movies-and-tv-shows")
print("Path to dataset files:", path)

print(os.listdir(path))

Path to dataset files: /root/.cache/kagglehub/datasets/shivamb/disney-movies-and-tv-shows/versions/2
['disney_plus_titles.csv']


In [110]:
df_disney_raw = pd.read_csv(os.path.join(path + "/disney_plus_titles.csv"))
df_disney_raw.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",NaN,"November 26, 2021",2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!
1,s2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",NaN,"November 26, 2021",1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. ...
2,s3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,"November 26, 2021",2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.
3,s4,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",NaN,"November 26, 2021",2021,TV-PG,41 min,Musical,"This is real life, not just fantasy!"
4,s5,TV Show,The Beatles: Get Back,NaN,"John Lennon, Paul McCartney, George Harrison, ...",NaN,"November 25, 2021",2021,NaN,1 Season,"Docuseries, Historical, Music",A three-part documentary from Peter Jackson ca...


## Tratamento de Dados

### Estatísticas

In [111]:
df_disney_raw.describe()


,release_year
count,1450.000000
mean,2003.091724
std,21.860162
min,1928.000000
25%,1999.000000
50%,2011.000000
75%,2018.000000
max,2021.000000


In [112]:
df_disney_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       1450 non-null   object
 1   type          1450 non-null   object
 2   title         1450 non-null   object
 3   director      977 non-null    object
 4   cast          1260 non-null   object
 5   country       1231 non-null   object
 6   date_added    1447 non-null   object
 7   release_year  1450 non-null   int64 
 8   rating        1447 non-null   object
 9   duration      1450 non-null   object
 10  listed_in     1450 non-null   object
 11  description   1450 non-null   object
dtypes: int64(1), object(11)
memory usage: 136.1+ KB


In [113]:
# Valores distintos na coluna 'show_id'
num_id = df_disney_raw['show_id'].nunique()
print(f"Há {num_id} valores distintos na coluna show_id")

Há 1450 valores distintos na coluna show_id


## Limpeza e Tratamento dos Dados

In [114]:
df_disney = df_disney_raw.copy()

In [115]:
# coluna de primary key
df_disney['id_pk'] = df_disney.index + 1

In [117]:
# Quantidade de valores nulos nas colunas do Dataset
df_disney.isnull().sum()

,0
show_id,0
type,0
title,0
director,473
cast,190
country,219
date_added,3
release_year,0
rating,3
duration,0


In [118]:
# Quantidade de linhas duplicadas
df_disney.duplicated().sum()

np.int64(0)

In [119]:
# Substitui valores nulos (NaN) por desconhecido (Unknow)
df_disney['director'].fillna('Unknown', inplace=True)
df_disney['cast'].fillna('Unknown', inplace=True)
df_disney['country'].fillna('Unknown', inplace=True)
df_disney['rating'].fillna('Unrated', inplace=True)


/tmp/ipython-input-3023197969.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_disney['director'].fillna('Unknown', inplace=True)
/tmp/ipython-input-3023197969.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', t

In [120]:
df_disney.head(20)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,id_pk
0,s1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",Unknown,"November 26, 2021",2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!,1
1,s2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",Unknown,"November 26, 2021",1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. ...,2
2,s3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,"November 26, 2021",2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.,3
3,s4,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",Unknown,"November 26, 2021",2021,TV-PG,41 min,Musical,"This is real life, not just fantasy!",4
4,s5,TV Show,The Beatles: Get Back,Unknown,"John Lennon, Paul McCartney, George Harrison, ...",Unknown,"November 25, 2021",2021,Unrated,1 Season,"Docuseries, Historical, Music",A three-part documentary from Peter Jackson ca...,5
5,s6,Movie,Becoming Cousteau,Liz Garbus,"Jacques Yves Cousteau, Vincent Cassel",United States,"November 24, 2021",2021,PG-13,94 min,"Biographical, Documentary",An inside look at the legendary life of advent...,6
6,s7,TV Show,Hawkeye,Unknown,"Jeremy Renner, Hailee Steinfeld, Vera Farmiga,...",Unknown,"November 24, 2021",2021,TV-14,1 Season,"Action-Adventure, Superhero",Clint Barton/Hawkeye must team up with skilled...,7
7,s8,TV Show,Port Protection Alaska,Unknown,"Gary Muehlberger, Mary Miller, Curly Leach, Sa...",United States,"November 24, 2021",2015,TV-14,2 Seasons,"Docuseries, Reality, Survival",Residents of Port Protection must combat volat...,8
8,s9,TV Show,Secrets of the Zoo: Tampa,Unknown,"Dr. Ray Ball, Dr. Lauren Smith, Chris Massaro,...",United States,"November 24, 2021",2019,TV-PG,2 Seasons,"Animals & Nature, Docuseries, Family",A day in the life at ZooTampa is anything but ...,9
9,s10,Movie,A Muppets Christmas: Letters To Santa,Kirk R. Thatcher,"Steve Whitmire, Dave Goelz, Bill Barretta, Eri...",United States,"November 19, 2021",2008,G,45 min,"Comedy, Family, Musical",Celebrate the holiday season with all your fav...,10


In [121]:
def extrair_unidade_e_valor(duration_str):
        if pd.isna(duration_str):
            return (None, None)
        duration_str = str(duration_str).strip()

        unidade = None
        if 'min' in duration_str:
            unidade = 'min'
        elif 'Season' in duration_str:
            unidade = 'Season'
        elif 'Seasons' in duration_str:
            unidade = 'Seasons'
        else:
            return (None, None)

        match = re.search(r'(\d+)', duration_str)
        valor = int(match.group(1)) if match else None

        return (unidade, valor)

# Criar as novas colunas
df_disney[['duration_unit', 'duration_value']] = df_disney['duration'].apply(
        lambda x: pd.Series(extrair_unidade_e_valor(x))
    )
df_disney.rename(columns={'type': 'content_type'}, inplace=True)
df_disney.drop(columns=['duration', 'description'], inplace=True)

colunas_finais = [
        'id_pk', 'content_type', 'title', 'director', 'cast', 'country',
        'date_added', 'release_year', 'rating', 'duration_value',
        'duration_unit', 'listed_in'
    ]
df_disney_sql = df_disney[colunas_finais].copy()

In [122]:
df_disney_sql.head(20)

,id_pk,content_type,title,director,cast,country,date_added,release_year,rating,duration_value,duration_unit,listed_in
0,1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",Unknown,"November 26, 2021",2016,TV-G,23,min,"Animation, Family"
1,2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",Unknown,"November 26, 2021",1988,PG,91,min,Comedy
2,3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,"November 26, 2021",2011,TV-G,23,min,"Animation, Comedy, Family"
3,4,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",Unknown,"November 26, 2021",2021,TV-PG,41,min,Musical
4,5,TV Show,The Beatles: Get Back,Unknown,"John Lennon, Paul McCartney, George Harrison, ...",Unknown,"November 25, 2021",2021,Unrated,1,Season,"Docuseries, Historical, Music"
5,6,Movie,Becoming Cousteau,Liz Garbus,"Jacques Yves Cousteau, Vincent Cassel",United States,"November 24, 2021",2021,PG-13,94,min,"Biographical, Documentary"
6,7,TV Show,Hawkeye,Unknown,"Jeremy Renner, Hailee Steinfeld, Vera Farmiga,...",Unknown,"November 24, 2021",2021,TV-14,1,Season,"Action-Adventure, Superhero"
7,8,TV Show,Port Protection Alaska,Unknown,"Gary Muehlberger, Mary Miller, Curly Leach, Sa...",United States,"November 24, 2021",2015,TV-14,2,Season,"Docuseries, Reality, Survival"
8,9,TV Show,Secrets of the Zoo: Tampa,Unknown,"Dr. Ray Ball, Dr. Lauren Smith, Chris Massaro,...",United States,"November 24, 2021",2019,TV-PG,2,Season,"Animals & Nature, Docuseries, Family"
9,10,Movie,A Muppets Christmas: Letters To Santa,Kirk R. Thatcher,"Steve Whitmire, Dave Goelz, Bill Barretta, Eri...",United States,"November 19, 2021",2008,G,45,min,"Comedy, Family, Musical"


In [123]:
df_disney_sql.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id_pk           1450 non-null   int64 
 1   content_type    1450 non-null   object
 2   title           1450 non-null   object
 3   director        1450 non-null   object
 4   cast            1450 non-null   object
 5   country         1450 non-null   object
 6   date_added      1447 non-null   object
 7   release_year    1450 non-null   int64 
 8   rating          1450 non-null   object
 9   duration_value  1450 non-null   int64 
 10  duration_unit   1450 non-null   object
 11  listed_in       1450 non-null   object
dtypes: int64(3), object(9)
memory usage: 136.1+ KB


In [124]:
df_disney_sql['date_added'] = pd.to_datetime(df_disney_sql['date_added'])
df_disney_sql['duration_value'] = df_disney['duration_value'].astype(float)
df_disney_sql.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id_pk           1450 non-null   int64         
 1   content_type    1450 non-null   object        
 2   title           1450 non-null   object        
 3   director        1450 non-null   object        
 4   cast            1450 non-null   object        
 5   country         1450 non-null   object        
 6   date_added      1447 non-null   datetime64[ns]
 7   release_year    1450 non-null   int64         
 8   rating          1450 non-null   object        
 9   duration_value  1450 non-null   float64       
 10  duration_unit   1450 non-null   object        
 11  listed_in       1450 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(8)
memory usage: 136.1+ KB


In [125]:
df_disney_sql.describe()

,id_pk,date_added,release_year,duration_value
count,1450.000000,1447,1450.000000,1450.000000
mean,725.500000,2020-05-22 01:54:26.620594176,2003.091724,52.753793
min,1.000000,2019-10-01 00:00:00,1928.000000,1.000000
25%,363.250000,2019-11-12 00:00:00,1999.000000,4.000000
50%,725.500000,2019-11-12 00:00:00,2011.000000,46.000000
75%,1087.750000,2020-11-26 00:00:00,2018.000000,93.000000
max,1450.000000,2021-11-26 00:00:00,2021.000000,183.000000
std,418.723258,NaN,21.860162,46.557763


In [126]:
df_disney_sql.head(20)

,id_pk,content_type,title,director,cast,country,date_added,release_year,rating,duration_value,duration_unit,listed_in
0,1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",Unknown,2021-11-26,2016,TV-G,23.0,min,"Animation, Family"
1,2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",Unknown,2021-11-26,1988,PG,91.0,min,Comedy
2,3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,2021-11-26,2011,TV-G,23.0,min,"Animation, Comedy, Family"
3,4,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",Unknown,2021-11-26,2021,TV-PG,41.0,min,Musical
4,5,TV Show,The Beatles: Get Back,Unknown,"John Lennon, Paul McCartney, George Harrison, ...",Unknown,2021-11-25,2021,Unrated,1.0,Season,"Docuseries, Historical, Music"
5,6,Movie,Becoming Cousteau,Liz Garbus,"Jacques Yves Cousteau, Vincent Cassel",United States,2021-11-24,2021,PG-13,94.0,min,"Biographical, Documentary"
6,7,TV Show,Hawkeye,Unknown,"Jeremy Renner, Hailee Steinfeld, Vera Farmiga,...",Unknown,2021-11-24,2021,TV-14,1.0,Season,"Action-Adventure, Superhero"
7,8,TV Show,Port Protection Alaska,Unknown,"Gary Muehlberger, Mary Miller, Curly Leach, Sa...",United States,2021-11-24,2015,TV-14,2.0,Season,"Docuseries, Reality, Survival"
8,9,TV Show,Secrets of the Zoo: Tampa,Unknown,"Dr. Ray Ball, Dr. Lauren Smith, Chris Massaro,...",United States,2021-11-24,2019,TV-PG,2.0,Season,"Animals & Nature, Docuseries, Family"
9,10,Movie,A Muppets Christmas: Letters To Santa,Kirk R. Thatcher,"Steve Whitmire, Dave Goelz, Bill Barretta, Eri...",United States,2021-11-19,2008,G,45.0,min,"Comedy, Family, Musical"


### Criando tabela 2 tabelas de acordo com o tipo de stream (Movie or TV Show)

In [128]:
df_disney_movie_sql = df_disney_sql[df_disney_sql['content_type'] == 'Movie']
df_disney_serie_sql = df_disney_sql[df_disney_sql['content_type'] == 'TV Show']

In [130]:
df_disney_movie_sql.head(15)

,id_pk,content_type,title,director,cast,country,date_added,release_year,rating,duration_value,duration_unit,listed_in
0,1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",Unknown,2021-11-26,2016,TV-G,23.0,min,"Animation, Family"
1,2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",Unknown,2021-11-26,1988,PG,91.0,min,Comedy
2,3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,2021-11-26,2011,TV-G,23.0,min,"Animation, Comedy, Family"
3,4,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",Unknown,2021-11-26,2021,TV-PG,41.0,min,Musical
5,6,Movie,Becoming Cousteau,Liz Garbus,"Jacques Yves Cousteau, Vincent Cassel",United States,2021-11-24,2021,PG-13,94.0,min,"Biographical, Documentary"
9,10,Movie,A Muppets Christmas: Letters To Santa,Kirk R. Thatcher,"Steve Whitmire, Dave Goelz, Bill Barretta, Eri...",United States,2021-11-19,2008,G,45.0,min,"Comedy, Family, Musical"
10,11,Movie,Adventure Thru the Walt Disney Archives,John Gleim,"Don Hahn, Kathryn Beaumont, Pete Docter, Kevin...",Unknown,2021-11-19,2020,TV-PG,59.0,min,Documentary
11,12,Movie,Puppy for Hanukkah,Unknown,Unknown,Unknown,2021-11-19,2020,TV-G,4.0,min,"Comedy, Family, Music"
12,13,Movie,The Pixar Story,Leslie Iwerks,"Stacy Keach, John Lasseter, Brad Bird, John Mu...",United States,2021-11-19,2007,G,91.0,min,"Documentary, Family"
14,15,Movie,America the Beautiful,Unknown,Michael B. Jordan,Unknown,2021-11-12,2021,TV-PG,2.0,min,Animals & Nature


In [129]:
df_disney_serie_sql.head(10)

,id_pk,content_type,title,director,cast,country,date_added,release_year,rating,duration_value,duration_unit,listed_in
4,5,TV Show,The Beatles: Get Back,Unknown,"John Lennon, Paul McCartney, George Harrison, ...",Unknown,2021-11-25,2021,Unrated,1.0,Season,"Docuseries, Historical, Music"
6,7,TV Show,Hawkeye,Unknown,"Jeremy Renner, Hailee Steinfeld, Vera Farmiga,...",Unknown,2021-11-24,2021,TV-14,1.0,Season,"Action-Adventure, Superhero"
7,8,TV Show,Port Protection Alaska,Unknown,"Gary Muehlberger, Mary Miller, Curly Leach, Sa...",United States,2021-11-24,2015,TV-14,2.0,Season,"Docuseries, Reality, Survival"
8,9,TV Show,Secrets of the Zoo: Tampa,Unknown,"Dr. Ray Ball, Dr. Lauren Smith, Chris Massaro,...",United States,2021-11-24,2019,TV-PG,2.0,Season,"Animals & Nature, Docuseries, Family"
13,14,TV Show,"Dr. Oakley, Yukon Vet",Unknown,"Dr. Michelle Oakley, Zachary Fine",United States,2021-11-17,2013,TV-PG,10.0,Season,"Action-Adventure, Animals & Nature, Docuseries"
17,18,TV Show,Disney Fancy Nancy,Unknown,"Mia Jenness, Alyson Hannigan, Rob Riggle, Spen...","United States, Canada",2021-11-12,2018,TV-PG,3.0,Season,"Animation, Kids"
18,19,TV Show,Disney Intertwined,Unknown,"Carolina Domenech, José Jiménez Zapiola, Kevsh...",Unknown,2021-11-12,2021,TV-PG,1.0,Season,"Comedy, Coming of Age, Drama"
28,29,TV Show,Olaf Presents,Unknown,Josh Gad,Unknown,2021-11-12,2021,TV-PG,1.0,Season,"Animation, Family"
51,52,TV Show,Disney Amphibia,Unknown,"Justin Felbinger, Amanda Leighton, Bill Farmer...","United States, South Korea",2021-11-03,2018,TV-Y7,3.0,Season,"Animation, Comedy, Family"
52,53,TV Show,Photo Ark,Unknown,Joel Sartore,United States,2021-11-03,2017,TV-PG,1.0,Season,"Animals & Nature, Docuseries, Family"


## Carregamento dos dados SQLite

In [131]:
DISNEY_DB = 'disney_etl_project.db'
DISNEY_TABLE = 'conteudo_disney'

conn = sqlite3.connect(DISNEY_DB)

print(f"\nCarregando dados na tabela '{DISNEY_TABLE}' do banco '{DISNEY_DB}'...")

df_disney_sql.to_sql(DISNEY_TABLE, conn, if_exists='replace', index=False)

conn.execute(f"CREATE UNIQUE INDEX idx_id_pk ON {DISNEY_TABLE} (id_pk);")

conn.commit()
conn.close()

print("Carregamento e Criação do Banco SQLite concluídos com sucesso.")


Carregando dados na tabela 'conteudo_disney' do banco 'disney_etl_project.db'...
Carregamento e Criação do Banco SQLite concluídos com sucesso.


## Consulta SQL

In [138]:
# --- 3. EXECUTANDO CONSULTAS SQL ---
print("\n--- 3. EXECUTANDO CONSULTAS SQL ---")

DISNEY_DB = 'disney_etl_project.db'
DISNEY_TABLE = 'conteudo_disney'

try:
    conn = sqlite3.connect(DISNEY_DB)

    # Consulta 1: Filmes COM DURAÇÃO > 90min e de diretores populares
    query_1 = f"""
    WITH DiretoresPopulares AS (
        -- Identifica diretores com 5 ou mais Filmes e duração média > 90 min
        SELECT
            director
        FROM
            {DISNEY_TABLE}
        WHERE
            content_type = 'Movie'
            AND duration_unit = 'min'
            AND director <> 'Unknown' -- Ignora diretores desconhecidos
        GROUP BY
            director
        HAVING
            COUNT(id_pk) >= 5
            AND AVG(duration_value) > 90
    )

    SELECT
        T1.title,
        T1.director,
        T1.release_year,
        T1.duration_value AS runtime_minutos
    FROM
        {DISNEY_TABLE} AS T1
    JOIN
        DiretoresPopulares AS T2 ON T1.director = T2.director
    WHERE
        T1.content_type = 'Movie'
    ORDER BY
        T1.director, T1.release_year DESC
    LIMIT 10;
    """
    print("\n[Consulta 1] Top 10 dos filmes longos com diretores populares:")
    print(pd.read_sql_query(query_1, conn))


    # Consulta 2: Tendência de Lançamento por Gênero (Foco em Animação)
    query_2 = f"""
    SELECT
        release_year,
        content_type,
        COUNT(id_pk) AS total_titulos_ano,
        SUM(CASE WHEN listed_in LIKE '%Animation%' THEN 1 ELSE 0 END) AS total_animacao,
        -- Calcula o percentual de títulos de animação
        ROUND(CAST(SUM(CASE WHEN listed_in LIKE '%Animation%' THEN 1 ELSE 0 END) AS REAL) * 100 / COUNT(id_pk), 2) AS percentual_animacao
    FROM
        {DISNEY_TABLE}
    WHERE
        release_year >= 2010
    GROUP BY
        release_year, content_type
    ORDER BY
        release_year DESC, content_type;
    """
    print("\n[Consulta 2] Tendência de Títulos Lançados (Pós-2010) com Foco em Animação:")
    print(pd.read_sql_query(query_2, conn))

    # --- CONSULTA 3: Distribuição de Duração por Classificação (Rating) ---
    query_3 = f"""
    SELECT
        rating,
        COUNT(id_pk) AS total_filmes,
        ROUND(AVG(duration_value), 2) AS duracao_media_minutos,
        MIN(duration_value) AS duracao_minima_minutos,
        MAX(duration_value) AS duracao_maxima_minutos
    FROM
        {DISNEY_TABLE}
    WHERE
        content_type = 'Movie'
        AND duration_unit = 'min'
        AND release_year >= 2010 -- Focando em filmes mais recentes
    GROUP BY
        rating
    HAVING
        COUNT(id_pk) > 5 -- Exclui classificações com poucas amostras
    ORDER BY
        duracao_media_minutos DESC;
    """
    print("\n[Consulta 3] Duração Média de Filmes (Pós-2010) por Classificação Indicativa:")
    print(pd.read_sql_query(query_3, conn))

  # --- Consulta 4: Programas de TV com MAIOR número de Temporadas ---

    query_max_seasons = f"""
    SELECT
        title,
        duration_value AS total_seasons,
        release_year,
        listed_in AS generos
    FROM
        {DISNEY_TABLE}
    WHERE
        content_type = 'TV Show'
        AND duration_unit = 'Season' -- Garante que estamos olhando para temporadas e não minutos
    ORDER BY
        duration_value DESC -- Ordena do maior para o menor
    LIMIT 5; -- Mostra os 5 programas de TV com mais temporadas (pode haver empate)
    """
    print("\n[Consulta 4] Programas de TV com MAIOR Número de Temporadas:")
    print(pd.read_sql_query(query_max_seasons, conn))


    # --- Consulta 5: Programas de TV com MENOR número de Temporadas ---
    # Usamos uma subconsulta para garantir que o menor valor encontrado é o 1 (o valor mínimo esperado).
    query_min_seasons = f"""
    SELECT
        title,
        duration_value AS total_seasons,
        release_year,
        listed_in AS generos
    FROM
        {DISNEY_TABLE}
    WHERE
        content_type = 'TV Show'
        AND duration_unit = 'Season'
        AND duration_value IS NOT NULL
        AND duration_value = (
            -- Subconsulta para encontrar o valor mínimo real (que deve ser 1)
            SELECT MIN(duration_value)
            FROM {DISNEY_TABLE}
            WHERE content_type = 'TV Show' AND duration_unit = 'Season'
        )
    ORDER BY
        release_year DESC -- Ordena por ano de lançamento para ver os mais recentes
    LIMIT 5; -- Mostra 5 exemplos de programas com o número mínimo de temporadas
    """
    print("\n[Consulta 5] Programas de TV com MENOR Número de Temporadas:")
    print(pd.read_sql_query(query_min_seasons, conn))

    conn.close()

except sqlite3.OperationalError as e:
    print(f"ERRO DE CONEXÃO: Certifique-se de que a Célula 2 (ETL) foi executada com sucesso. Erro: {e}")


--- 3. EXECUTANDO CONSULTAS SQL ---

[Consulta 1] Top 10 dos filmes longos com diretores populares:
                                               title        director  \
0                                      Incredibles 2       Brad Bird   
1                                       Tomorrowland       Brad Bird   
2                                        Ratatouille       Brad Bird   
3                                   Jack-Jack Attack       Brad Bird   
4                                    The Incredibles       Brad Bird   
5  Percy Jackson & The Olympians: The Lightning T...  Chris Columbus   
6                                     Mrs. Doubtfire  Chris Columbus   
7                     Home Alone 2: Lost in New York  Chris Columbus   
8                                         Home Alone  Chris Columbus   
9                          Adventures in Babysitting  Chris Columbus   

   release_year  runtime_minutos  
0          2018            118.0  
1          2015            131.0  
2